# Описание задачи
Это практическая работа по парсингу различных сервисов с помощью библиотек  requests, BeautifulSoup и Selenium. Нужно получить некоторые данные и сохранить их.

### BeautifulSoup
Соберите информацию со всех страниц о происходивших выставках кошек с сайта RU-pets.ru (http://ru-pets.ru/index.php?m=6&c=2&to=1).

Данные, которые необходимы:

- Дата проведения
- Название выставки
- Клуб-Организатор

Результат необходимо записать в CSV файл

In [154]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd

result = []
max_page=1

try:
    response = requests.get('http://ru-pets.ru/index.php?m=6&to=1&c=2')
    soup = BeautifulSoup(response.text, 'lxml') 
    max_page = int(soup
                   .find('a', attrs={'title':'Выставки кошек, последняя страница.'})
                   .get_text())

    if max_page:
        print('Всего страниц:', max_page)
        
    for page in range(1, max_page+1):
        response = (requests.get('http://ru-pets.ru/index.php?m=6&to=1&c=2',
                                 params={'page': page}))
        soup = BeautifulSoup(response.text, 'lxml') 
        exhibitions = soup.find_all('div', attrs={'class':'listitem'})
        
        for exhibition in exhibitions:
            name = exhibition.find('span', class_='cl-green')
            if name:
                name = name.get_text().strip()
            else:
                name = ''

            date = exhibition.find('h2')
            if date:
                date = date.get_text().strip().split(',')[0]
            else:
                date = ''

            host = exhibition.find('div', class_='msgtext').get_text().split('\n')
            if len(host) > 1:
                host = (host[1]
                        .replace('Клуб - Организатор: ', '')
                        .replace(';', '')
                        .strip())
            else:
                host = ''
            result.append({'name':name, 'date':date, 'host':host})
                   
    with open('cat_exhibitions.csv', 'w', encoding='utf-8') as result_file:
        field_names = ['name', 'date', 'host']
        writer = csv.DictWriter(result_file, fieldnames=field_names)
        writer.writeheader()
        for row in result:
            writer.writerow(row)
except Exception as ex:
    print(ex)
    
df = pd.read_csv('cat_exhibitions.csv', encoding='utf-8')
df.head()

Всего страниц: 77


,name,date,host
0,«КОТОШЕСТВИЕ»,22-23 октября 2022г.,КЛК Параллель
1,«ЛЕТО - 2022»,12-13 июня 2022г.,Самарис
2,«КЭТ-САЛОН-ОКТЯБРЬ»,23 октября 2021г.,КЛК Москва
3,«РОСКОШНАЯ ОСЕНЬ»,9-10 октября 2021г.,КЛК РосКош
4,«ОСЕННИЙ ВЕРНИСАЖ-2021»,2-3 октября 2021г.,УРОФО Грация


### Selenium
Напишите код, который с помощью Selenium соберет характеристики ноутбука (https://www.wildberries.ru/catalog/169234407/detail.aspx)

Информация должна быть собрана в формате JSON

In [ ]:
from selenium import webdriver
import csv
import time
import json
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


URL = 'https://www.wildberries.ru/catalog/169234407/detail.aspx'
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20)
action = ActionChains(driver)

try:
    driver.maximize_window()
    driver.get(URL)
    time.sleep(3)
    locator = (By.XPATH, "//button[contains(@data-link,'Все характеристики и описание')]")
    driver.find_element(*locator).click()
 
    popup = wait.until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "div.popup.popup-product-details.shown"))
    )

    product_data = {}
    category_blocks = popup.find_elements(By.CSS_SELECTOR, 'table.product-params__table')
    
    for block in category_blocks:
        caption = block.find_element(By.CSS_SELECTOR, 'caption').text.strip()
        product_data[caption] = {}

        rows = block.find_elements(By.CSS_SELECTOR, 'tr.product-params__row')
        for row in rows:
            key = row.find_element(By.CSS_SELECTOR, 'th.product-params__cell').text.strip()
            value = row.find_element(By.CSS_SELECTOR, 'td.product-params__cell').text.strip()
            product_data[caption][key] = value
            
    description_section = popup.find_element(By.CSS_SELECTOR, 'section.product-details__description')
    description = description_section.find_element(By.CSS_SELECTOR, 'p.option__text').text.strip()
    
    product_data['Описание'] = description
    
    with open('specs.json', 'w', encoding='utf-8') as f:
        json.dump(product_data, f)
    
except Exception as ex:
    print(ex)
    driver.quit()
finally:
    driver.quit()

### Рестораны с ZOON.ru
Необходимо собрать данные о ресторанах Томска с сайта ZOON.ru

Необходимо собрать следующие данные:

- Название заведения
- Рейтинг заведения
- Направление или направления заведения (пиццерия, бар и.тд)
 
Данные должны быть собраны в CSV-файл, чтобы передать его для дальнейшего анализа.

In [ ]:
from selenium import webdriver
import undetected_chromedriver as uc
import csv
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

URL = 'https://zoon.ru/tomsk/restaurants/'
driver = uc.Chrome()
wait = WebDriverWait(driver, timeout=10)
action = ActionChains(driver)
html = ''

try:    
    driver.get(URL)
    driver.implicitly_wait(100)
    driver.maximize_window()
    
    while True:
        button_next = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "js-next-page")))
        if button_next:
            driver.execute_script("arguments[0].scrollIntoView();", button_next)
            cards = driver.find_elements(By.CLASS_NAME, 'minicard-item__info')
            if len(cards)>0:
                for card in cards:
                    html += card.get_attribute('outerHTML')
            driver.execute_script("arguments[0].click();", button_next)
            time.sleep(random.uniform(1, 3))
        if not button_next:
            break
            
except Exception as ex:
    print(ex)
    driver.quit()
finally:
    driver.quit()

with open('tomsk-restaurants.html', 'w', encoding='utf-8') as f:
    f.write(html)

In [ ]:
from bs4 import BeautifulSoup
import csv

file = open('tomsk-restaurants.html', 'r', encoding='utf-8')
html = file.read()
file.close()

soup = BeautifulSoup(html, 'lxml')
cards = soup.find_all('div', class_='minicard-item__info')
data = []

def get_card_data(card):
    name = card.find('h2', class_='minicard-item__title').get_text().strip()
    rating = card.find('div', class_='z-text--bold').get_text().strip()
    cafe_type = (card
                 .find('div', class_='minicard-item__features')
                 .get_text()
                 .strip()
                 .replace('\n', '')
                 .replace('•', ', ')[:-2]
                )
    return {
        'name':name,
        'rating':rating,
        'cafe_type':cafe_type,
    }

for card in cards:
    card_data = get_card_data(card)
    data.append(card_data)

if len(data) > 0:
    with open('tomsk-restaurants.csv', 'w', encoding='utf-8') as f:
        fieldnames = data[0].keys()
        dict_writer = csv.DictWriter(f, fieldnames=fieldnames)
        dict_writer.writeheader()
        dict_writer.writerows(data)